<a href="https://colab.research.google.com/github/Xornotor/Food101-CNN-Tensorflow/blob/main/Food101_CNN_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, SeparableConv2D, Conv2D
from tensorflow.keras.layers import Flatten, Dense, AvgPool2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization, Normalization, Dropout
from tensorflow.keras.layers import Reshape, Resizing, Softmax
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import tensorflow_datasets as tfds
from random import randint

from google.colab import drive
if(not os.path.exists('/content/drive')):
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_ds, train_info = tfds.load('food101',
                            split='train',
                            with_info=True,
                            shuffle_files=True,
                            as_supervised=True,
                            batch_size=1024
                          )

val_ds, val_info = tfds.load('food101',
                             split='validation',
                             with_info=True,
                             shuffle_files=True,
                             as_supervised=True,
                             batch_size=1024
                          )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incompleteMI5NVS/food101-train.tfrecord*...:   0%|          | 0/…

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incompleteMI5NVS/food101-validation.tfrecord*...:   0%|         …

Dataset food101 downloaded and prepared to ~/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [5]:
DROPOUT = 0.5
REGULARIZATION = 1e-4

def create_model():
  model = Sequential([
      Input((None, None, 3)),
      Resizing(256, 256, 'bilinear', True),
      Normalization(),
      SeparableConv2D(6, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(12, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(32, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(64, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      SeparableConv2D(128, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(256, 3, (2, 2), padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      SeparableConv2D(384, 3, padding='same',
                      activation='relu',
                      kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      AvgPool2D((2, 2)),
      Flatten(),
      Dense(192, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(128, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(101, activation='relu', kernel_regularizer=L2(REGULARIZATION)),
      Dropout(DROPOUT),
      BatchNormalization(),
      Dense(101, activation='linear', kernel_regularizer=L2(REGULARIZATION)),
      BatchNormalization(),
      Softmax()
  ])

  model.compile(optimizer=RMSprop(4e-3),
                loss=SparseCategoricalCrossentropy(from_logits=False),
                metrics=[SparseCategoricalAccuracy()])
  
  return model

In [6]:
#FoodNet = create_model()
#FoodNet.summary()

In [7]:
model_path = '/content/drive/MyDrive/Colab Notebooks/FoodNet/FoodNet.h5'
#model_path = '/content/FoodNet.h5'

cp_callback = ModelCheckpoint(filepath=model_path,
                            save_weights_only=False,
                            verbose=1)

if(os.path.exists(model_path)):
    FoodNet = load_model(model_path)
    pass

FoodNet.fit(train_ds,
            epochs=15,
            validation_data=val_ds,
            callbacks=[cp_callback])


Epoch 1/15
74/74 [==============================] - ETA: 0s - loss: 3.6528 - sparse_categorical_accuracy: 0.1672
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/FoodNet/FoodNet.h5
74/74 [==============================] - 218s 3s/step - loss: 3.6528 - sparse_categorical_accuracy: 0.1672 - val_loss: 3.6261 - val_sparse_categorical_accuracy: 0.1747
Epoch 2/15
74/74 [==============================] - ETA: 0s - loss: 3.6064 - sparse_categorical_accuracy: 0.1741
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/FoodNet/FoodNet.h5
74/74 [==============================] - 201s 3s/step - loss: 3.6064 - sparse_categorical_accuracy: 0.1741 - val_loss: 3.5659 - val_sparse_categorical_accuracy: 0.1865
Epoch 3/15
74/74 [==============================] - ETA: 0s - loss: 3.5664 - sparse_categorical_accuracy: 0.1800
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/FoodNet/FoodNet.h5
74/74 [==============================] - 201s 3s/step - loss: 3.5664 - spars